In [2]:
import numpy as np
import pandas as pd

data = pd.read_csv("wandb_export_2024-11-14T10_01_51.979+01_00.csv")
finished_data = data[data['State'] == 'finished']

def get_mean_std(dt):
    if dt.size == 0:
        return np.nan, np.nan
    else:
        return dt.mean(), dt.std()
    

UCI = ['boston-housing', 'concrete', 'energy', 'kin8nm', 'naval-propulsion-plant', 'power-plant', 'wine-quality-red', 'yacht']
ls = []

for dataset in UCI:
    result = {}
    data_class = finished_data[(finished_data['dataset'] == dataset)]

    # homoscedastic
    homo_data = data_class[(data_class['likelihood'] == 'homoscedastic') & (data_class['method'] == 'marglik')]
    homo_pp = np.array(homo_data['test/loglik_bayes'])
    result['homo'] = get_mean_std(homo_pp)

    naive_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'map')]
    naive_lik = np.array(naive_data['test/loglik'])
    result['naive_gs'] = get_mean_std(naive_lik)

    # Other methods
    betahalf_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'betanll') & (data_class['beta'] == 0.5)]
    betahalf_lik = np.array(betahalf_data['test/loglik'])
    result['betahalf'] = get_mean_std(betahalf_lik)

    betaone_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'betanll') & (data_class['beta'] == 1)]
    betaone_lik = np.array(betaone_data['test/loglik'])
    result['betaone'] = get_mean_std(betaone_lik)

    faith_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'faithful')]
    faith_lik = np.array(faith_data['test/loglik'])
    result['faith'] = get_mean_std(faith_lik)
    
    mcdropout_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'mcdropout')]
    mcdropout_lik = np.array(mcdropout_data['test/loglik'])
    result['mcdropout'] = get_mean_std(mcdropout_lik)

    vi_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['method'] == 'vi')]
    vi_lik = np.array(vi_data['test/loglik'])
    result['vi'] = get_mean_std(vi_lik)

    # naive
    naive_eb_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'marglik')]
    naive_eb_pp_lik = np.array(naive_eb_pp_data['test/loglik_bayes'])
    result['naive_eb_pp'] = get_mean_std(naive_eb_pp_lik)

    naive_eb_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'meanvar') & (data_class['method'] == 'marglik')]
    naive_eb_lik = np.array(naive_eb_data['test/loglik'])
    result['naive_eb'] = get_mean_std(naive_eb_lik)

    # natural
    natural_gs_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'map')]
    natural_gs_pp_lik = np.array(natural_gs_pp_data['test/loglik_bayes'])
    result['natural_gs_pp'] = get_mean_std(natural_gs_pp_lik)

    natural_gs_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'map')]
    natural_gs_lik = np.array(natural_gs_data['test/loglik'])
    result['natural_gs'] = get_mean_std(natural_gs_lik)

    natural_eb_pp_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'marglik')]
    natural_eb_pp_lik = np.array(natural_eb_pp_data['test/loglik_bayes'])
    result['natural_eb_pp'] = get_mean_std(natural_eb_pp_lik)

    natural_eb_data = data_class[(data_class['likelihood'] == 'heteroscedastic') & (data_class['head'] == 'natural') & (data_class['method'] == 'marglik')]
    natural_eb_lik = np.array(natural_eb_data['test/loglik'])
    result['natural_eb'] = get_mean_std(natural_eb_lik)

    ls.append(result)


# Define the table columns
objective = ["Homoscedastic", "Naive NLL", "β-NLL (0.5)", "β-NLL (1.0)", 
             "Faithful", "MC-Dropout", "VI", "Naive NLL", "Naive NLL", 
             "Natural NLL", "Natural NLL", "Natural NLL", "Natural NLL"]
regularization = ["EB", "GS", "GS", "GS", "GS", "GS", "GS", "EB", "EB", 
                  "GS", "GS", "EB", "EB"]
posterior_predictive = ["Y", "N", "N", "N", "N", "N", "N", "Y", "N", "Y", "N", "Y", "N"]

# Initialize DataFrame
table_updated = pd.DataFrame(columns=["Objective", "Regularization", "Posterior Predictive"] + UCI)
table_updated["Objective"] = objective
table_updated["Regularization"] = regularization
table_updated["Posterior Predictive"] = posterior_predictive

# Populate the table with values from `ls`
model_configs = ["homo", "naive_gs", "betahalf", "betaone", "faith", 
                 "mcdropout", "vi", "naive_eb_pp", "naive_eb", 
                 "natural_gs_pp", "natural_gs", "natural_eb_pp", "natural_eb"]

for i, config in enumerate(model_configs):
    for j, dataset in enumerate(UCI):
        mean, std = ls[j].get(config, (np.nan, np.nan))
        if not np.isnan(mean) and not np.isnan(std):
            table_updated.loc[i, dataset] = f"{mean:.2f} ({std:.2f})"
        else:
            table_updated.loc[i, dataset] = "N/A"

# Print the table
print(table_updated.to_string(index=False))

table_updated.to_csv("results_table.csv")




    Objective Regularization Posterior Predictive boston-housing     concrete       energy      kin8nm naval-propulsion-plant  power-plant wine-quality-red          yacht
Homoscedastic             EB                    Y   -2.64 (0.24) -3.04 (0.12) -0.71 (0.18) 1.29 (0.03)            5.62 (1.69) -2.80 (0.04)     -0.96 (0.09)   -1.08 (0.75)
    Naive NLL             GS                    N   -2.79 (0.23) -3.32 (0.22) -2.20 (2.17) 1.34 (0.05)         -14.25 (23.68) -2.84 (0.14)     -0.94 (0.08) -10.40 (11.74)
  β-NLL (0.5)             GS                    N   -2.75 (0.34) -3.35 (0.32) -1.17 (0.44) 1.31 (0.04)            4.52 (2.06) -2.89 (0.22)     -0.97 (0.09)   -1.18 (1.63)
  β-NLL (1.0)             GS                    N   -2.77 (0.36) -3.19 (0.16) -1.13 (0.40) 1.28 (0.03)            6.88 (0.36) -2.87 (0.15)     -0.93 (0.08)   -2.73 (2.43)
     Faithful             GS                    N   -3.30 (0.39) -3.72 (0.08) -2.28 (0.10) 0.19 (0.02)            5.13 (0.08) -2.92 (0.03)     -1